# **Análise Exploratória de Dados**

In [ ]:
!pip install sidetable

Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sidetable as stb

from sklearn.preprocessing import OneHotEncoder
warnings.filterwarnings("ignore")


Carga dos dados

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/seguros/train.csv')
test=pd.read_csv('/content/drive/MyDrive/seguros/test.csv')

Cópia do dataframe train

In [ ]:
data=train.copy()

In [ ]:
data.head()

Renomeando as colunas

In [ ]:
#lista com novos nomes das colunas
nomes=['id','sexo','idade','habilitacao','cod_regiao','prev_seguro','veiculo_idade','veiculo_danos','taxa_anual','canal_vendas','tempo_filiado','resposta']



In [ ]:
#dicionario os nomes da colunas
colunas=dict(zip(data.columns.to_list(),nomes))


In [ ]:
#renomeando as 
data.rename(columns=colunas,inplace=True)

Infomaçãos do dataframe

In [ ]:
data.info()

In [ ]:
desc       = ["ID unico para o cliente.",
              "Sexo do cliente.",
              "Idade do cliente.",
              "0: não tem habilitacao, 1: tem habilitacao.",
              "Codigo unico da regiao do cliente.",
              "1 : Cliente já possui seguro, 0 : Cliente não possui seguro.",
              "Idade do Veiculo.",
              "1 : Cliente ja teve veiculo danificado no passado. 0 : Cliente não teve veiculo danificado no passado.",
              "O valor que o cliente precisa pagar como taxa no ano.",
              "Código para o canal de divulgação ao cliente ou seja. Agentes diferentes, por correio, por telefone, pessoalmente, etc..",
              "Número de dias, o cliente foi associado à empresa.",
              "1 : Cliente esta interessado, 0 : Cliente não esta interessadpo"]

data_desc = []
j = 0
for column in data.columns:
  data_desc.append(
      [
       column,
       data[column].dtypes,
       data[column].isnull().sum(),
       round(data[column].isnull().sum()/len(data)*100, 2),
       data[column].nunique(),
       data[column].unique(),
       desc[j]
       ]
       )
  j += 1

df = pd.DataFrame(data_desc, columns = ['Coluna', 'Dtype', 'Null', 'Null (%)', 'Nunique', 'Unique','Descrição'])
df



Registros duplicados

In [ ]:
train.duplicated().sum()


Verificando balanceamento dos Dados

In [ ]:
g=sns.countplot(data=data,x='resposta')
for i in g.patches:
  g.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
  
# Apenas 12% dos clientes estão interessados ​​em comprar um seguro de veículo


# Análise Univariada

Analise descritiva 

In [ ]:
#seleciona variveis numericas e categoricas
var_num=data.select_dtypes(exclude='object')
var_cat=data.select_dtypes(include='object')

In [ ]:
# Media, Mediana
tcm = pd.DataFrame( var_num.apply( np.mean ) ).T
tcmd= pd.DataFrame( var_num.apply( np.median ) ).T

#  std, min, max, range, skew, kurtosis
dvp = pd.DataFrame( var_num.apply( np.std ) ).T 
v_min = pd.DataFrame( var_num.apply( min ) ).T 
v_max = pd.DataFrame( var_num.apply( max ) ).T 
range = pd.DataFrame( var_num.apply( lambda x: x.max() - x.min() ) ).T 
sk= pd.DataFrame( var_num.apply( lambda x: x.skew() ) ).T 
k = pd.DataFrame( var_num.apply( lambda x: x.kurtosis() ) ).T 

# concatenar
df = pd.concat( [v_min, v_max, range, tcm, tcmd, dvp, sk, k] ).T.reset_index()
df.columns = ['Variaveis', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
df
#A média de dias de clientes na empresa é de 154 dias
# nenhum cliente está na seguradora há 1 ano inteiro
# os clientes mais antigos neste conjunto de dados são 85 e a mediana é 36
# o valor maximo da taxa anual varia 94,13% comparado ao valor media no da taxa_anual


In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(3,2,1)
plt.title('Idade')
sns.distplot(data['idade'])

plt.subplot(3,2,2)
plt.title('Codigo da Regiao')

sns.histplot(data['cod_regiao'])

plt.subplot(3,2,3)
plt.title('Taxa Anual')

sns.distplot(data['taxa_anual'])

plt.subplot(3,2,4)
plt.title('Tempo Filiação em dias')

sns.histplot(data['tempo_filiado'])

plt.subplot(3,2,5)
plt.title('Canal de Vendas')

sns.histplot(data['canal_vendas'],bins=25)

plt.subplots_adjust(left=0.1, 
                    bottom=0.1,  
                    right=0.9,  
                    top=0.9,  
                    wspace=0.4,  
                    hspace=0.4) 




In [ ]:
var_cat.describe()
# ha mais clientes do sexo masculino
# A maioria do clientes tem um veículo com mais de um ano e menos de dois anos
# a maioria dos clientes neste conjunto de dados teve seu veículo danificado antes


In [ ]:
#Variavel geração baseada nas idades
# Baseado em (https://rockcontent.com/br/blog/dossie-das-geracoes/)
  
    # Millenials : 18 - 34
    # Gen X : 35 - 50
    # Boomer : 51 - 69
    # Silent : 70 e acima


faixa_idade = [18, 34, 50, 69, data['idade'].max()]
rotulo_geracao = ['Millenials', 'Gen X', 'Boomer', 'Silent']

data['geracao'] = pd.cut(data['idade'], bins = faixa_idade, labels = rotulo_geracao)
data.head()


Verificando outliers

In [ ]:
plt.figure(figsize = (15, 10))
x = 1      

for column in var_num:
    plt.subplot(3,3,x)
    sns.boxplot(data[column])
    x += 1
# podemos ver que há muitos outliers na coluna taxa_anual

In [ ]:
fig, ax = plt.subplots(figsize=(20,15))

plt.subplot(3,2,1)
plt.title('Sexo do Cliente')
ax=sns.countplot(data=data,x='sexo')
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
  
plt.subplot(3,2,2)
plt.title('Idade do Veiculo')
ax=sns.countplot(data=data,x='veiculo_idade')
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
  
plt.subplot(3,2,3)
plt.title('Danos no Veiculo')

ax=sns.countplot(data=data,x='veiculo_danos')
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
  
plt.subplot(3,2,4)
plt.title('Habilitação')
ax =sns.countplot(data=data,x='habilitacao')
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
plt.subplot(3,2,5)
plt.title('Previamente Asegurado')
ax =sns.countplot(data=data,x='prev_seguro')
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
  
plt.subplot(3,2,6)
plt.title('Geração')
ax =sns.countplot(data=data,x='geracao')
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
  
plt.subplots_adjust(left=0.1, 
                    bottom=0.1,  
                    right=0.9,  
                    top=0.9,  
                    wspace=0.4,  
                    hspace=0.4) 
# 54% do cliente do seguro saúde é do sexo masculino
# um pouco mais da metade dos clientes não possui seguro de veículo
# 4% dos clientes têm veículo com mais de 2 anos
# metade dos clientes já teve seu veículo danificado e metade não
# A maior parte dos clientes pertencem a millenials



In [ ]:
data['idade'].mean()
#idade média dos clientes é de cerca de 38 a 39 anos



In [ ]:
data.stb.freq(['cod_regiao'], cum_cols=False)
# código de região 28 tem o maior número de clientes enquando o codigo 52 possui o menor



In [ ]:
data.stb.freq(['canal_vendas'], cum_cols=False)
# canal de vendas 152,26,124 são os que possuem uma taxa de sucesso mais efetiva em realizar vendas
# os demais canais ficam abaixo de 5% de resposta efetiva


# **Analise Multivariada**


In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Sexo x Resposta',fontsize=20)
ax=sns.countplot(data['sexo'], hue = data['resposta'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')


In [ ]:
pd.crosstab(index = data['resposta'], columns = data['sexo'], normalize = 'index')
#Homens possuem mais interesse em contratar seguro de automovel do que mulheres

In [ ]:
data[['resposta','idade']].groupby(['resposta']).mean()

In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Idade x  Resposta',fontsize=20)
sns.boxplot(x=data['resposta'], y = data['idade'])


In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Faixa de Idade x  Resposta',fontsize=20)
x_idade=data.copy()
x_idade['faixa_idade']=data['idade'].apply(lambda x: 'ate_30' if x<30 else 'ate_50' if x>=30 and x<50 else 'maior_50')
x_idade=x_idade[['faixa_idade','resposta']].groupby(['faixa_idade','resposta']).size().reset_index().rename(columns={0:'qtd'})
ax=sns.barplot(data=x_idade,x='faixa_idade',y='qtd',hue='resposta');
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
total=sum(x_idade['qtd'])

# idade média do cliente interessado no seguro do veículo é de 43 anos
# Podemos ver que clientes mais jovens não estão interessados ​​no seguro do veículo

In [ ]:
pd.pivot_table(data,index=["resposta","habilitacao"],values=["idade"],aggfunc=np.mean)
# Os clientes que não possuem habilitação tem em media idade acima dos 60 anos
# Não ha tantos dados sobre não habilitados porque a maioria deles é acima dos 60 anos e a maioria dos clientes da base esta na faixa de 40 anos


In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Habilitação x Resposta',fontsize=20)
ax=sns.countplot(data['habilitacao'], hue = data['resposta'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')

In [ ]:
pd.crosstab(index = data['cod_regiao'], columns = data['resposta'], normalize = 'index').sort_values(1,ascending = False).head(5)


resposta,0,1
cod_regiao,,
38.0,0.807996,0.192004
28.0,0.812837,0.187163
19.0,0.837134,0.162866
4.0,0.841755,0.158245
23.0,0.846939,0.153061


In [ ]:
# região 28 tem o maior número de clientes e com maior precentuas de clientes interessados
# A regiao 38 possui mais clientes interessados se comparada com a regiao 28
# As regiao com melhor percentual de resposta utilizam os canais de venda mais eficientes

plt.figure(figsize = (18, 5))
plt.title('Codigo Região x Resposta',fontsize=20)
ax=sns.countplot(data['cod_regiao'], hue = data['resposta'])
ax.set_xticklabels(ax.get_xticks(), rotation = 90) 


In [ ]:
pd.crosstab(index = data['cod_regiao'], columns = data['resposta'], normalize = 'index').sort_values(1, ascending = False).head()


In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Codigo Região x Idade x Resposta',fontsize=20)
ax=sns.violinplot(data = data,x = 'cod_regiao', y = 'idade', hue = 'resposta', split = True, linewidth = 1)
ax.set_xticklabels(ax.get_xticks(), rotation = 90) 
ax.legend(loc = 2,bbox_to_anchor = (1,1))


In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Codigo Região x Canal de Vendas',fontsize=20)
ax=sns.violinplot(data = data,x = 'cod_regiao', y = 'canal_vendas', split = True, linewidth = 1)
ax.set_xticklabels(ax.get_xticks(), rotation = 90) 
ax.legend(loc = 2,bbox_to_anchor = (1,1))


In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Codigo Região x Canal de Vendas x Resposta',fontsize=20)
ax=sns.violinplot(data = data,x = 'cod_regiao', y = 'canal_vendas', hue = 'resposta', split = True, linewidth = 1)
ax.set_xticklabels(ax.get_xticks(), rotation = 90) 
ax.legend(loc = 2,bbox_to_anchor = (1,1))


In [ ]:
pd.crosstab(index = data['resposta'], columns = data['prev_seguro'], normalize = 'columns')

In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Previamente Segurado x Resposta',fontsize=20)
ax=sns.countplot(data['prev_seguro'], hue = data['resposta'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
  # clientes que ja possuem seguro de automovel mostram não tem interesse em adquirir um novo seguro
  # dos clientes que não possuem seguro apenas 12% se mostraram interessados em adquirir um seguro de automovel

In [ ]:
pd.crosstab(index = data['prev_seguro'], columns = data['resposta'], normalize = 'columns')


In [ ]:
data[['prev_seguro','idade']].groupby(['prev_seguro']).mean()
#Clientes mais jovens provavelmente ja posssuem um seguro de automovel

In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Idade x Previamente Segurado x Resposta',fontsize=20)
x_idade=data.copy()
x_idade['faixa_idade']=data['idade'].apply(lambda x: 'ate_30' if x<30 else 'ate_50' if x>=30 and x<50 else 'maior_50')
x_idade=x_idade[['faixa_idade','prev_seguro']].groupby(['faixa_idade','prev_seguro']).size().reset_index().rename(columns={0:'qtd'})
ax=sns.barplot(data=x_idade,x='faixa_idade',y='qtd',hue='prev_seguro');
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
total=sum(x_idade['qtd'])

In [ ]:
plt.figure(figsize = (18, 5))
x_idade=data.copy()
x_idade['faixa_idade']=data['idade'].apply(lambda x: 'ate_30' if x<30 else 'ate_50' if x>=30 and x<50 else 'maior_50')
x_idade=x_idade[['faixa_idade','resposta']].groupby(['faixa_idade','resposta']).size().reset_index().rename(columns={0:'qtd'})
x_idade=x_idade[x_idade['resposta']==1]
ax=sns.barplot(data=x_idade,x='faixa_idade',y='qtd');
for p in ax.patches:
  height = p.get_height()
  ax.text(p.get_x()+p.get_width()/2.,
                height,
                '{:1.2f}'.format(height/total*100),
                ha="center") 

total=sum(x_idade['qtd'])
#cerca de  87% dos clientes interessados tem mais que 30 anos de idade



In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Idade Veiculo x Resposta',fontsize=20)
ax=sns.countplot(data['veiculo_idade'], hue = data['resposta'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
#clientes que possuem veículo com mais de 2 anos tem maior probabilidade de se interessar pelo seguro do veículo


In [ ]:
pd.crosstab(index = data['prev_seguro'], columns = data['veiculo_idade'], normalize = 'columns')
#Quanto mais novo o veículo, maior a probabilidade de estar segurado


In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Idade Veiculo x Previamente Segurado',fontsize=20)
ax=sns.countplot(data['veiculo_idade'], hue = data['prev_seguro'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')

In [ ]:
pd.crosstab(index = data['resposta'], columns = [data['veiculo_idade'], data['prev_seguro']], normalize = 'columns')


In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Danos no veiculo x Resposta',fontsize=20)
ax=sns.countplot(data['veiculo_danos'], hue = data['resposta'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')


In [ ]:
pd.crosstab(index = data['resposta'], columns = data['veiculo_danos'], normalize = 'index')
# Dos clientes que tiveram danos no veiculo 98% tem interesse em contratar seguro


In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Danos no veiculo x Sexo',fontsize=20)
ax=sns.countplot(data['veiculo_danos'], hue = data['sexo'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
ax.legend(loc = 1)


 #homens tem mais danos em veiculos do que mulheres 

In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Danos no veiculo x Previamente segurado',fontsize=20)
ax=sns.countplot(data['veiculo_danos'], hue = data['prev_seguro'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
ax.legend(loc = 1)
#clientes que tiveram seu veículo danificado anteriormente não possuem seguro do veículo
# 87% dos clientes que nunca tiveram nenhum veículo danificado possuem seguro de veículo
# Pessoas que têm seguro de veículo são mais propensas a ter cuidado com seu veículo


In [ ]:
pd.crosstab(index = data['prev_seguro'], columns = data['veiculo_danos'], normalize = 'columns')


In [ ]:
pd.crosstab(data.veiculo_idade, data.veiculo_danos, normalize = 'index')
# Carros com mais de 2 anos são mais propensos a ter sofrido algum dano

In [ ]:
plt.figure(figsize = (18, 5))
plt.title('Idade do Veiculo x Danos no veiculo',fontsize=20)
ax=sns.countplot(data['veiculo_idade'], hue = data['veiculo_danos'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')

In [ ]:
pd.crosstab(index = data['resposta'], columns = [data['veiculo_danos'], data['prev_seguro']], normalize = 'index')
#dos clientes interessados a maioria deles não tem seguro do veículo e teve danos no veículo no passado
#clientes que nunca tiveram danos no veículo e possuem seguro de não  estão interessado em outro seguro de veículo


In [ ]:
plt.figure(figsize=(20,10))
sns.kdeplot(data=data, x = 'taxa_anual', hue = 'resposta',multiple='fill')
           
    

In [ ]:
data[['resposta','taxa_anual']].groupby(['resposta']).median()

In [ ]:
data[['veiculo_idade','taxa_anual']].groupby(['veiculo_idade']).median().plot(kind='bar',figsize=(15,8))
#A taxa anual media maior para veiculos com mais de 2 anos

In [ ]:
plt.figure(figsize=(15,8))
sns.kdeplot(data=data, x = 'tempo_filiado', hue = 'resposta',multiple='fill')

In [ ]:
data[['resposta','tempo_filiado']].groupby(['resposta']).mean()
# Tempo de filiacao não interfere na resposta do cliente

In [ ]:
plt.figure(figsize = (14, 7))
ax=sns.countplot(data['geracao'], hue = data['resposta'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
 # A geraçao GenX e Boomer tem maior interesse por seguros

In [ ]:
pd.crosstab(index = data['geracao'], columns = [data['veiculo_danos'], data['resposta']], normalize = 'columns')
#Dos clientes que tiveram danos a Gen X é mais interessadas em seguros

In [ ]:
pd.crosstab(index = data['geracao'], columns = data['veiculo_danos'], normalize = 'index')
# GenX e Boomer são as que mais tem veiculos que sofreram danos

In [ ]:
plt.figure(figsize=(15,8))
sns.violinplot(x=data['geracao'], y = data['taxa_anual'],split=True)


In [ ]:
plt.figure(figsize = (14, 5))
ax=sns.countplot(x = data['prev_seguro'], hue = data['geracao'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')



In [ ]:
pd.crosstab(index = data['geracao'], columns = data['prev_seguro'], normalize = 'index')
#A geração do milênio 62,7% já possuem previamente seguro de automovel
# mostra que a geração X 72% não possuem previamente seguro de automovel


In [ ]:
plt.figure(figsize = (14, 5))
ax=sns.countplot(x = data['veiculo_idade'], hue = data['geracao'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
 # A maioria da geracao millenials  tem veiculos com menos de 1 ano de idade


In [ ]:
plt.figure(figsize = (14, 5))
ax=sns.countplot(x = data['veiculo_danos'], hue = data['geracao'])
for i in ax.patches:
  ax.annotate((i.get_height()/len(data)*100).round(2),
             (i.get_x() + i.get_width()/2,i.get_height()),
             ha='center',va='baseline',fontsize=12,color='black',
             xytext=(0,1),textcoords='offset points')
# A geracao x é que maior porcentagem de veiculos com danos

In [ ]:
plt.figure(figsize = (10,10))
plt.title("Correlação")
corr_num= var_num.corr()
sns.heatmap(corr_num,annot=True);


# **Overview dos Dados**

* Apenas 12% dos clientes estão interessados ​​em comprar um seguro de veículo
* A média de dias  em que um clientes  se filiou na empresa é de 154 dias
* Não ha tantos dados sobre não habilitados porque a maioria deles é acima dos 60 anos e a maioria dos clientes da base esta na faixa de 40 anos
* Homens possuem mais interesse em contratar seguro de automovel do que mulheres
* canal de vendas 152,26,124 são os que possuem uma taxa de sucesso mais efetiva em realizar vendas
* os demais canais ficam abaixo de 5% de resposta efetiva
* código de região 28 tem o maior número de clientes enquando o codigo 52 possui o menor
* A regiao 38 possui mais clientes interessados se comparada com a regiao 28
* idade média do cliente interessado no seguro do veículo é de 43 anos
* Tempo de filiacao não interfere na resposta do cliente talvez porque nenhum cliente está na seguradora há 1 ano inteiro.
* dos clientes que não possuem seguro apenas 12% se mostraram interessados em adquirir um seguro de automovel
* dos clientes interessados a maioria deles não tem seguro do veículo 
* 98% cliente dos que tiveram danos no veículo no passado tem interesse em contratar seguro
* 87% dos clientes que nunca tiveram nenhum veículo danificado e possuem seguro  podemos dizer que elas são mais propensas a ter cuidado com seu veículo
* Quanto mais novo o veículo, maior a probabilidade de estar segurado
* 4% dos clientes têm veículo com mais de 2 anos,e a taxa anual media é maior para esse tipo de veiculo e estes são mais propensos a ter sofrido algum dano. Os clientes que possuem esse tipo de veiculo tem maior probabilidade de se interessar pelo seguro.
* A geracao x é que maior porcentagem de veiculos com danos ,e 72% não possuem previamente seguro de automovel e é mais interessadas em seguros.
* A maioria da geracao millenials  tem veiculos com menos de 1 ano de idade e 62,7% já possuem previamente seguro de automovel e a maioria dos cliente pertencem a essa geração.


